## BBR v2.0
Startup阶段面对丢包的操作：直接进入Drain
公平性：与Reno和Cubic等等共存的时候，减少ProbeBW阶段的侵略性
侵略性体现在探测带宽时过度发包导致缓存积压
改进：指数增长先慢后快地探测带宽

总的来说，是让模型更加综合。

## 更复杂的模型
从单一指标到复合指标，诊断网络中出现的问题。
 | | 时延	| 时延方差 |	带宽 |	带宽方差 |	其它... |
| --- | --- | --- | --- | --- | --- |
|稳定拥塞 |	大	 | 小 |	N/A |	N/A	| |
| 突发拥塞	| 大 |	大 |	N/A |	N/A	 |
| 随机丢包(高)	| 小 |	小	| 小 |	大	 |
| 随机丢包(低)	| 小 |	小	| 大 |	小	 |
| 没有问题 | | | | |

![图1](https://pic4.zhimg.com/v2-066af4e63ea78688d1ea7901fa9e66cd_1440w.jpg "摘自")

![续](https://picx.zhimg.com/v2-1eba656596c93dbd866aa59d244c5fd5_1440w.jpg)

```
    if n > 500 and n <= 1000:
        # 随机突发拥塞
        Buff = random.randint(0, 20)
    elif n > 1500 and n <= 2000:
        # 持续拥塞
        Buff = 20
    elif n > 2500 and n <= 3000:
        # 逐渐拥塞
        Buff += 0.04
    elif n > 3500 and n <= 4000:
        # 随机高丢包率(无拥塞)
        Buff = 0
        if random.random() < drate:
            wx[n] = wx[n]/2
            wy[n] = wy[n]/2
    elif n > 4200 and n <= 4600:
        # 随机低丢包率(无拥塞)
        Buff = 0
        if random.random() < drate/50:
            wx[n] = wx[n]/2
    else:
        # Good
        Buff = 0

    while wx[n] + wy[n] + Buff > 1.2*C*R:
        wx[n] = wx[n]/2
        wy[n] = wy[n]/2
```